In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt


In [ ]:
dir_SLAND = '/Trendy/Data/SLAND_Trendy-v10_S2_countries/'
dir_ctrs  = '/Trendy/Data/data_ancillary/info_countries/'
dir_fig   = '/GCB2021_commentary/Figures/'


## Country definitions and time

In [ ]:
#Read countries
fname_cntrs = dir_ctrs + 'Country codes 3 letters.xlsx'
data_cntrs  = pd.read_excel(fname_cntrs, sheet_name=0, header=None, index_col=0)

#Define country lists
countries     = ['USA', 'RUS', 'CAN', 'CHN', 'BRA', 'IDN', 'COD']
ctrs_EU       = ['AUT', 'BEL', 'BGR', 'HRV', 'CYP', 'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'IRL', 'ITA', 'LVA', 'LTU', 'LUX', 'MLT', 'NLD', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP', 'SWE', 'GBR']
countries_all = countries + ctrs_EU
countries_sel = ['USA', 'RUS', 'EU27_UK', 'CAN', 'CHN', 'BRA', 'IDN', 'COD']

#Define time
time_sta = 2001
time_end = 2015
time_len = time_end - time_sta + 1


## Read SLAND

In [ ]:
#Define models
models_SLAND = ['CABLE-POP', 'CLASSIC', 'CLM5.0', 'DLEM', 'IBIS', 'ISAM', 'ISBA-CTRIP', 'JSBACH', 'JULES-ES-1.1',
                'LPJ-GUESS', 'LPJwsl', 'LPX-Bern', 'OCN', 'ORCHIDEEv3', 'SDGVM', 'VISIT', 'YIBs']#'CLASSIC-N', 'ORCHIDEE'

#Define parameters to select datasets
selection     = 'non-intact-forests'
data_sets     = ['DGVMs-Forests-PFTs','DGVMs-NaturalLand-PFTs']#, 'BLUE']

#Dictionary for storing data
data_SLAND = dict()

#Loop over datasets
for data_set in data_sets:

    #Define different forest cover thresholds according to Hansen et al. (2013)
    if data_set=='DGVMs-Forests-PFTs':         Hansen_treshs = [0.05, 0.10, 0.15, 0.20, 0.25]#, 0.30, 0.35]
    elif data_set=='DGVMs-NaturalLand-PFTs':   Hansen_treshs = [0.15]    
    
    #Loop over forest cover thresholds
    for Han2013_thresh in Hansen_treshs:

        
        #Define file name
        fname = dir_SLAND + 'SLAND-S2-countries_' + selection + '_Weights-' + data_set + '_MaxForCover' + '{:.2f}'.format(Han2013_thresh) + '_vRemapToForestMask-Hansen2010_IFL_2013.xlsx'

        #Create empyt dataframes
        data_SLAND_coll = pd.DataFrame()

        #Loop over models
        data_sum = np.zeros(len(models_SLAND))
        for i, model in enumerate(models_SLAND):

            #Read SLAND data
            data_SLAND_read = pd.read_excel(fname, sheet_name=model + '_SLAND_IPCC_ctrs', header=0, index_col=0)

            #Select time
            data_SLAND_read = data_SLAND_read.loc[(data_SLAND_read.index>=time_sta) & (data_SLAND_read.index<=time_end)]

            #Check time selection
            if (data_SLAND_read.index[0]!=time_sta) | (data_SLAND_read.index[-1]!=time_end) | (len(data_SLAND_read.index)!=time_len):
                sys.exit('Check time selection of SLAND!')
                
            #Calculate time average
            data_SLAND_read = data_SLAND_read.mean(axis=0)
            
            #Calculate global sum
            data_sum[i] = data_SLAND_read.sum()
            
            #Select countries
            data_SLAND_read = data_SLAND_read[countries_all]

            #Calculate SLAND in EU27&UK
            data_SLAND_read['EU27_UK'] = data_SLAND_read[ctrs_EU].sum()
            data_SLAND_read = data_SLAND_read.drop(ctrs_EU)

            #Set correct sign vor SLAND
            data_SLAND_coll[model] = -data_SLAND_read

        #Save in dictionary
        name_out = data_set + '_' + '{:.2f}'.format(Han2013_thresh)
        data_SLAND[name_out] = data_SLAND_coll
        
        print('{:.2f}'.format(Han2013_thresh), end=': ')
        print('{:.2f}'.format(np.mean(data_sum) * 44 / 12 / 1000) + ' Pg CO2 / year')
    
    print('')

## UNITS
#data_SLAND has units: Tg C / year


## Plot

In [ ]:
#Create figure and add second axis
fig, ax = plt.subplots(1, 1, figsize=(18 * len(countries_sel)/10, 5))

#Define tresholds that should be plotted
Hansen_treshs = [0.05, 0.10, 0.15, 0.20, 0.25]

#Loop over countries
ctrs  = []
p_all = []
for i, country in enumerate(countries_sel):
    
    print('')
    print(country, end=': ')

    #Loop over forest cover thresholds
    for k, Han2013_thresh in enumerate(Hansen_treshs):

        #Extract for forest PFTs and convert to Pg CO2 / year
        data_extract = data_SLAND['DGVMs-Forests-PFTs_' + '{:.2f}'.format(Han2013_thresh)]
        data_extract = data_extract.loc[country] / 1e3 * 44 / 12

        #Calculate multi-model median
        data_plot = data_extract.median()

        #Define x and y values
        x = [i + 1/3 * (0+1) -0.12 , i + 1/3 * (0+1) + 0.12]
        y = [data_plot, data_plot]

        #Define line width
        if Han2013_thresh==0.15:  lw = 4
        else:                     lw = 2        
        
        #Plot
        p = ax.plot(x, y, lw=lw, color='forestgreen', zorder=5)#, alpha=alpha[k])
        if i==0 and k==2:  p_all.append(p[0])

        #Write percentage values for forest fraction
        if country=='EU27_UK':
            if Han2013_thresh==0.05:  corr = -0.01
            else:                     corr = 0
            ax.text(x[0] - 0.05, y[0] + corr, '{:.0f}'.format(Han2013_thresh * 100) + '%', ha='right', va='center', fontsize=9)

    #Extract values for natural land PFTs
    data_extract = data_SLAND['DGVMs-NaturalLand-PFTs_0.15']
    data_extract = data_extract.loc[country] / 1e3 * 44 / 12

    #Calculate multi-model median
    data_plot = data_extract.median()

    #Define x and y values
    x = [i + 1/3 * (1+1) -0.12 , i + 1/3 * (1+1) + 0.12]
    y = [data_plot, data_plot]                                  
        
    #Plot
    p = ax.plot(x, y, lw=4, color='limegreen', zorder=5) 
    p_all.append(p[0])
                                               
    #Make stripes in background
    if np.mod(i, 2)==0:
        ax.axvspan(i, i + 1, facecolor='black', alpha=0.05)
        
    #Define country names
    if country=='USA':        ctr_name = country
    elif country=='RUS':      ctr_name = 'Russia'
    elif country=='COD':      ctr_name = 'DR Congo'
    elif country=='TZA':      ctr_name = 'Tanzania'
    elif country=='CAF':      ctr_name = 'Cent. Afr. Rep.'
    elif country=='EU27_UK':  ctr_name = 'EU27&UK'
    else:                     ctr_name = data_cntrs.loc[country].item()
    ctrs.append(ctr_name)
    
#Set x-ticks and x-tick labels
ax.set_xticks(np.arange(0.5, len(ctrs)))
ax.set_xticklabels(ctrs, rotation=0, ha='center', fontsize=14)

#Set limits and y-label
ax.set_xlim([0, len(ctrs)])
ax.set_ylim([-1, 0])
ax.set_ylabel('CO$_2$ flux / Pg CO$_2$ yr$^{-1}$', fontsize=20, labelpad=15)    

#Set xticks and size of ticks
ax.tick_params(axis='y', labelcolor='k')
ax.tick_params(labelsize=15)
ax.tick_params(axis='x', which='major', pad=10)
ax.tick_params(axis="x", which="both", bottom=False, top=False)

#Legend
legend = ax.legend(p_all, ['forest PFTs', 'natural land PFTs'], fontsize=18, loc=4, frameon=False)

#Save figure
fig.savefig(dir_fig + 'FigS1_natural-land-sink_different-weighting-factors.png', dpi=300, bbox_inches='tight')
